# RecSys M 2021 - Exercise 3 Template

The aims of this exercise are:
 - Explore a different recommendation dataset
 - Develop and evaluate baseline recommender systems
 - Implement hybrid recommender models
 - Explore diversification issues in recommender systems
 - Revise other material from the lectures.

As usual, there is a corresponding Quiz on Moodle for this Exercise, which should be answered as you proceed. For more details, see the Exercise 3 specification.



# Part-Pre. Preparation 

## Pre 1. Setup Block

This exercise will use the [Goodreads]() dataset for books. These blocks setup the data files, Python etc.

In [1]:
# !rm -rf ratings* books* to_read* test*

# !curl -o ratings.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-ratings.csv" 
# !curl -o books.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-books.csv"
# !curl -o to_read.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-to_read.csv"
# !curl -o test.csv "http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-test.csv"

In [1]:
#Standard setup
import pandas as pd
import numpy as np
import torch
# !pip install git+https://github.com/cmacdonald/spotlight.git@master#egg=spotlight
from spotlight.interactions import Interactions
SEED=20
BPRMF=None

## Pre 2. Data Preparation

Let's load the dataset into dataframes.

In [2]:
#load in the csv files
ratings_df = pd.read_csv("ratings.csv")
books_df = pd.read_csv("books.csv")
to_read_df = pd.read_csv("to_read.csv")
test = pd.read_csv("test.csv")

In [3]:
#cut down the number of items and users
counts=ratings_df[ratings_df["book_id"] < 2000].groupby(["book_id"]).count().reset_index()
valid_books=counts[counts["user_id"] >= 10][["book_id"]]

books_df = books_df.merge(valid_books, on="book_id")
ratings_df = ratings_df[ratings_df["user_id"] < 2000].merge(valid_books, on="book_id")
to_read_df = to_read_df[to_read_df["user_id"] < 2000].merge(valid_books, on="book_id")
test = test[test["user_id"] < 2000].merge(valid_books, on="book_id")


#stringify the id columns
def str_col(df):
  if "user_id" in df.columns:
    df["user_id"] = "u" + df.user_id.astype(str)
  if "book_id" in df.columns:
    df["book_id"] = "b" + df.book_id.astype(str)

str_col(books_df)
str_col(ratings_df)
str_col(to_read_df)
str_col(test)

Here we construct the Interactions objects from `ratings.csv`, `to_read.csv` and `test.csv`. We manually specify the num_users and num_items parameters to all Interactions objects, in case the test set differs from your training sets.

In [4]:
from collections import defaultdict
from itertools import count

from spotlight.cross_validation import random_train_test_split

iid_map = defaultdict(count().__next__)


rating_iids = np.array([iid_map[iid] for iid in ratings_df["book_id"].values], dtype = np.int32)
test_iids = np.array([iid_map[iid] for iid in test["book_id"].values], dtype = np.int32)
toread_iids = np.array([iid_map[iid] for iid in to_read_df["book_id"].values], dtype = np.int32)


uid_map = defaultdict(count().__next__)
test_uids = np.array([uid_map[uid] for uid in test["user_id"].values], dtype = np.int32)
rating_uids = np.array([uid_map[uid] for uid in ratings_df["user_id"].values], dtype = np.int32)
toread_uids = np.array([uid_map[iid] for iid in to_read_df["user_id"].values], dtype = np.int32)


uid_rev_map = {v: k for k, v in uid_map.items()}
iid_rev_map = {v: k for k, v in iid_map.items()}


rating_dataset = Interactions(user_ids=rating_uids,
                               item_ids=rating_iids,
                               ratings=ratings_df["rating"].values,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

toread_dataset = Interactions(user_ids=toread_uids,
                               item_ids=toread_iids,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

test_dataset = Interactions(user_ids=test_uids,
                               item_ids=test_iids,
                               num_users=len(uid_rev_map),
                               num_items=len(iid_rev_map))

print(rating_dataset)
print(toread_dataset)
print(test_dataset)

#here we define the validation set
toread_dataset_train, validation = random_train_test_split(toread_dataset, random_state=np.random.RandomState(SEED))

num_items = test_dataset.num_items
num_users = test_dataset.num_users

<Interactions dataset (1999 users x 1826 items x 124762 interactions)>
<Interactions dataset (1999 users x 1826 items x 135615 interactions)>
<Interactions dataset (1999 users x 1826 items x 33917 interactions)>


Finally, this is some utility code that we will use in the exercise.

In [5]:
def getAuthorTitle(iid):
  bookid = iid_rev_map[iid]
  row = books_df[books_df.book_id == bookid]
  return row.iloc[0]["authors"] + " / " + row.iloc[0]["title"]

print("iid 0: " + getAuthorTitle(0) )

iid 0: Carlos Ruiz Zafón, Lucia Graves / The Shadow of the Wind (The Cemetery of Forgotten Books,  #1)


## Pre 3. Example Code

To evaluate some of your hand-implemented recommender systems (e.g. Q1, Q4), you will need to instantiate objects that match the specification of a Spotlight model, which `mrr_score()` etc. expects.


Here is an example recommender object that returns 0 for each item, regardless of user.

In [6]:
from spotlight.evaluation import mrr_score, precision_recall_score

class dummymodel:
  
  def __init__(self, numitems):
    self.predictions=np.zeros(numitems)
  
  #uid is the user we are requesting recommendations for;
  #returns an array of scores, one for each item
  def predict(self, uid):
    #this model returns all zeros, regardless of userid
    return( self.predictions )

#lets evaluate how the effeciveness of dummymodel

print(mrr_score(dummymodel(num_items), test_dataset, train=rating_dataset, k=100).mean())
#as expected, a recommendation model that gives 0 scores for all items obtains a MRR score of 0

0.0


In [7]:
#note that mrr_score() displays a progress bar if you set verbose=True
print(mrr_score(dummymodel(num_items), test_dataset, train=rating_dataset, k=100, verbose=True).mean())


1999it [00:00, 4004.37it/s]

0.0


# Part-A. Combination of Recommendation Models

## Task 1. Explicit & Implicit Matrix Factorisation Models

Create and train three matrix factorisation systems:
 - "EMF": explicit MF, trained on the ratings Interactions object (`rating_dataset`)
 - "IMF": implicit MF, trained on the toread_dataset Interactions object (`toread_dataset_train`)
 - "BPRMF": implicit MF with the BPR loss function (`loss='bpr'`), trained on the toread_dataset Interactions object (`toread_dataset_train`)

Use a variable of the same name for these models, as we will use some of them later (e.g. `BPRMF`).
  
In all cases, you must use the standard initialisation arguments, i.e. 
`n_iter=10, embedding_dim=32, use_cuda=False, random_state=np.random.RandomState(SEED)`.
 
Evaluate each of these models in terms of Mean Reciprocal Rank on the test set. MRR can be obtained using:
```python
mrr_score(X, test_dataset, train=rating_dataset, k=100, verbose=True).mean())
```
where X is an instance of a Spotlight model. Do NOT change the `k` or `train` arguments.

In [8]:
# Add your solution here
import time
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.factorization.implicit import ImplicitFactorizationModel

# explicit MF, trained on the ratings Interactions object (rating_dataset)
EMF = ExplicitFactorizationModel(n_iter=10,
                                 embedding_dim=32,
                                 use_cuda=False,
                                 random_state=np.random.RandomState(SEED)) #20
current = time.time()
EMF.fit(rating_dataset)
end = time.time()
print("Training took %d seconds for EMF"% ((end - current)))
EMF_rr = mrr_score(EMF, test_dataset, train=rating_dataset, k=100, verbose=True)
print("MRR for EMF:", EMF_rr.mean())

# implicit MF, trained on the toread_dataset Interactions object (toread_dataset_train)
IMF = ImplicitFactorizationModel(n_iter=10, 
                                 embedding_dim=32,
                                 use_cuda=False,
                                 random_state=np.random.RandomState(SEED))
current = time.time()
IMF.fit(toread_dataset_train)
end = time.time()
print("Training took %d seconds for IMF"% ((end - current)))
IMF_rr = mrr_score(IMF, test_dataset, train=rating_dataset, k=100, verbose=True)
print("MRR for IMF:", IMF_rr.mean())

# implicit MF with the BPR loss function (loss='bpr'), trained on the toread_dataset Interactions object (toread_dataset_train)
BPRMF = ImplicitFactorizationModel(n_iter=10, 
                                   embedding_dim=32,
                                   use_cuda=False,
                                   random_state=np.random.RandomState(SEED),
                                   loss='bpr')
current = time.time()
BPRMF.fit(toread_dataset_train)
end = time.time()
print("Training took %d seconds for BPRMF"% ((end - current)))
BPR_rr = mrr_score(BPRMF, test_dataset, train=rating_dataset, k=100, verbose=True)
print("MRR for BPRMF:", BPR_rr.mean())


109it [00:00, 1084.83it/s]

Training took 17 seconds for EMF


1999it [00:01, 1195.52it/s]


MRR for EMF: 0.05898399982013507


112it [00:00, 1114.73it/s]

...

MRR for IMF: 0.3299315791285401


101it [00:00, 1000.32it/s]

Training took 28 seconds for BPRMF


1999it [00:01, 1126.50it/s]

MRR for BPRMF: 0.4076771464674879


## Task 2. Hybrid Model

In this task, you are expected to create new hybrid recommendation models that 
combine the two models in Task 1, namely IMF and BPRMF. 

(a) Linearly combine the *scores* from IMF and BPRMF.  Normalise both input scores into the range 0..1 using [sklearn's minmax_scale() function](
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html) before combining them.

(b) Apply a pipelining recommender, where the top 100 items are obtained from IMF and re-ranked using the scores of BPRMF. Items not returned by IMF get a score of 0.

To implement these hybrid models, you should create new classes that abide by the Spotlight model contract (namely, it has a `predict(self, uid)` function that returns a score for *all* items). 

Evaluate each model in terms of MRR. How many users are improved, how many are degraded compared to the BPRMF baseline?

Finally, pass your instantiated model object to the `test_Hybrid_a()` (for (a)) or `test_Hybrid_b()` (for (b)) functions, as appropriate, and record the results in the quiz. For example, if your model for (b) is called `pipeline`, then you would run:
```python
test_Hybrid_b(pipeline)
```

You now have sufficient information to answer the Task 2 quiz questions.

In [10]:
def test_Hybrid_a(combsumObj):
  for i, u in enumerate([5, 20]):
    print("Hybrid a test case %d" % i)
    print(np.count_nonzero(combsumObj.predict(u) > 1))

def test_Hybrid_b(pipeObj):
  for i, iid in enumerate([3, 0]):
    print("Hybrid b test case %d" % i)
    print(pipeObj.predict(0)[iid])


In [11]:
# Add your solutions here and evaluate them
from sklearn.preprocessing import minmax_scale

# for hybrid models
class hybridModel:
    def __init__(self, scores):
        self.predictions=scores
# uid is the user we are requesting recommendations for;
# returns an array of scores, one for each item
    def predict(self, uid):
    #this model returns a score of all items for each user
        return self.predictions[uid]

# scores for linear model
# normalised IMF and BPR scores
normal_IMF = minmax_scale([IMF.predict(i) for i in range(num_users)])
normal_BPR = minmax_scale([BPRMF.predict(i) for i in range(num_users)])
# linearly combine them
linear_model = hybridModel(normal_IMF + normal_BPR)
# evaluate how the effectiveness of linear model
linear_rr = mrr_score(linear_model, test_dataset, train=rating_dataset, k=100)
print(linear_rr.mean())


0.2617672345739065


In [12]:
# improved compared with BPR
print(sum(item > 0 for item in (linear_rr - BPR_rr)))
# degraded by linear
print(sum(item < 0 for item in (linear_rr - BPR_rr)))

582
1227


In [13]:
# scores for pipeline recommender
from scipy.stats import rankdata

pipe_score = []
for uid in range(num_users):
#     IMF and BPR score for a single user
    IMF_score = IMF.predict(uid)
    BPR_score = BPRMF.predict(uid)
#     rank IMF score to get top 100 items
    ranks = rankdata(IMF_score)
#     score = BPR_score if this item ranks top 100 otherwise score = 0
    rerank_score = [BPR_score[idx] if ranks[idx] > (num_items - 100) else 0 for idx in range(num_items)]
    pipe_score.append(rerank_score)
    
# pipeline model
pipe_model = hybridModel(np.array(pipe_score))
# evaluate how the effectiveness of pipeline
pipe_rr = mrr_score(pipe_model, test_dataset, train=rating_dataset, k=100)
print(pipe_rr.mean())

0.41511955311399246


In [14]:
# improved compared with BPR
print(sum(item > 0 for item in (pipe_rr - BPR_rr)))
# degraded by pipeline
print(sum(item < 0 for item in (pipe_rr - BPR_rr)))

586
214


In [15]:
#Now test your hybrid approaches for the quiz

test_Hybrid_a(linear_model)
test_Hybrid_b(pipe_model)


Hybrid a test case 0
800
Hybrid a test case 1
658
Hybrid b test case 0
22.01358413696289
Hybrid b test case 1
0.0


# Part-B. Analysing Recommendation Models

## Utility methods

Below, we provide a function, `get_top_K(model, uid : int, k : int)` which, when provided with a Spotlight model, will provide the top k predictions for the specified uid. The iids, their scores, and their embeddings are returned. 

In [16]:
from typing import Sequence, Tuple

def get_top_K(model, uid : int, k : int) -> Tuple[ Sequence[int], Sequence[float],  np.ndarray ] :
  #returns iids, their (normalised) scores in descending order, and item emebddings for the top k predictions of the given uid.

  from sklearn.preprocessing import minmax_scale

  from scipy.stats import rankdata
  # get scores from model
  scores = model.predict(uid)

  # map scores into rank 0..1 over the entire item space
  scores = minmax_scale(scores)

  #compute their ranks  
  ranks = rankdata(-scores)
  
  # get and filter iids, scores and embeddings
  rtr_scores = scores[ranks <= k]
  rtr_iids = np.argwhere(ranks <= k).flatten()
  if hasattr(model, '_net'):
    embs = model._net.item_embeddings.weight[rtr_iids]
  else:
    # not a model that has any embeddings
    embs = np.zeros([k,1])

  # identify correct ordering using numpy.argsort()
  ordering = (-1*rtr_scores).argsort()

  #return iids, scores and their embeddings in descending order of score
  return rtr_iids[ordering], rtr_scores[ordering], embs[ordering]

if BPRMF is not None:
  iids, scores, embs = get_top_K(BPRMF, 0, 10)
  print("Returned iids: %s" % str(iids))
  print("Returned scores: %s" % str(scores))
  print("Returned embeddings: %s" % str(embs))
else:
  print("You need to define BPRMF in Task 1")



Returned iids: [ 23 108  21  33   9  81  52 254  16   3]
Returned scores: [1.         0.98951113 0.9848312  0.92250615 0.9070821  0.9065416
 0.90052915 0.8930985  0.88378024 0.8836938 ]
Returned embeddings: tensor([[-0.0453,  1.3716, -0.8307, -1.2615,  1.6700,  1.0161,  1.1169,  2.3530,
         -1.2027,  0.8522, -1.0941, -0.6865, -0.5726, -2.0335, -1.2590,  0.6154,
         -0.1374, -1.6869, -1.8616, -0.7514,  1.9909, -0.3910,  1.9240,  1.3294,
         -1.2834, -0.4520,  1.1338,  0.3468,  2.5168, -2.1587,  1.2310,  1.1670],
        [ 0.1240,  1.1004,  0.0531, -1.1045,  1.9932,  1.5049,  1.0011,  1.9734,
         -1.6322, -0.8913, -0.6372,  0.7721, -1.1422, -2.2424, -1.1936, -0.5770,
          0.0762, -1.0283, -1.2807, -2.0889,  2.8154, -0.9600, -0.1419,  0.8408,
         -1.6067, -1.2906,  1.9169,  1.3988,  1.8646, -2.2029,  0.5365,  0.2022],
        [ 0.3844,  0.8188, -0.1892, -1.1793,  2.1731,  0.6669,  1.1271,  1.4538,
         -1.2173, -0.5447, -1.6714,  0.5249, -0.6132, -3.1083,

## Task 3. Evaluation of Non-personalised Models
Implement the following four (non-personalised) baselines for ranking books based on their statistics:
 - Average rating, obtained from ratings_df, `ratings` column
 - Number of ratings, obtained from books_df (column `ratings_count`)
 - Number of 5* ratings, obtained from books_df (column `ratings_5`)
 - Fraction of 5* ratings, calculated from the two sources of evidence above, i.e (columns  `ratings_5` and `ratings_count`).

Evaluate these in terms of MRR using the provided test data. You may use the StaticModel class below. 

Hints: 
 - As in Exercise 2, the order of items returned by predict() is _critical_. You may wish to refer to iid_map.
 - For all models, you need to ensure that your values are not cast to ints. If you are extracting values from a Pandas series, it is advised to use [.astype(np.float32)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.astype.html).


In [17]:
class StaticModel:
  
  def __init__(self, staticscores):
    self.numitems = len(staticscores)
    #print(self.numitems)
    assert isinstance(staticscores, np.ndarray), "Expected a numpy array"
    assert staticscores.dtype == np.float32 or staticscores.dtype == np.float64, "Expected a numpy array of floats"
    self.staticscores = staticscores
  
  def predict(self, uid):
    #this model returns the same scores for each user    
    return self.staticscores

In [18]:
# Add your solution here

# Average rating, obtained from ratings_df, ratings column
avg_ratings_origin = ratings_df.groupby(['book_id'])['rating'].mean().astype(np.float32)
# change the order to be the same as prediction
avg_ratings = [avg_ratings_origin[bid] for bid in iid_map.keys()]
avg_ratings_model = StaticModel(np.array(avg_ratings))
# evaluate the model in terms of MRR
print(mrr_score(avg_ratings_model, test_dataset, train=rating_dataset, k=100).mean())

0.015052024168984034


In [19]:
# Number of ratings, obtained from books_df (column ratings_count)
num_ratings = [books_df[books_df['book_id'] == bid]['ratings_count'].iloc[0].astype(np.float32) for bid in iid_map.keys()]
num_ratings_model = StaticModel(np.array(num_ratings))
# evaluate the model in terms of MRR
print(mrr_score(num_ratings_model, test_dataset, train=rating_dataset, k=100).mean())

0.2396001188245477


In [20]:
# Number of 5* ratings, obtained from books_df (column ratings_5)
num_5_ratings = [books_df[books_df['book_id'] == bid]['ratings_5'].iloc[0].astype(np.float32) for bid in iid_map.keys()]
num_5_ratings_model = StaticModel(np.array(num_5_ratings))
# evaluate the model in terms of MRR
print(mrr_score(num_5_ratings_model, test_dataset, train=rating_dataset, k=100).mean())

0.2409670879930144


In [21]:
# Fraction of 5* ratings, calculated from the two sources of evidence above, i.e (columns ratings_5 and ratings_count).
frac_5_ratings = np.array(num_5_ratings) / np.array(num_ratings)
frac_5_ratings_model = StaticModel(frac_5_ratings)
# evaluate the model in terms of MRR
print(mrr_score(frac_5_ratings_model, test_dataset, train=rating_dataset, k=100).mean())

0.03415267465103555


## Task 4. Qualiatively Examining Recommendations

From now on, we will consider the `BPRMF` model.

In Recommender Systems, the ground truth (i.e. our list of books that the user has added to their "to_read" shelf) can be very incomplete. For instance, this can be because the user is not aware of the book yet.

For this reason, it is important to "eyeball" the recommendations, to understand what the system is surfacing, and whether the recommendations make sense. In this way, we understand if the recommendations are reasonable, even if they are for books that the user has not actually read according to the test dataset.

First, write a function, which given a uid (int), prints the *title and authors* of:
 - (a) the books that the user has previously shelved (c.f. `toread_dataset`)
 - (b) the books that the user will read in the future (c.f. `test_dataset`)
 - (c) the top 10 books that the user were recommended by `BPRMF` - you can make use of `get_top_K()`.

You can use the previously defined `getAuthorTitle()` function in your solution.
You will also want to compare books in (c) with those in (a) and (b).

Then, we will examine two specific users, namely uid 1805 (u336) and uid 179 (user u1331), to analyse if their recommendations make sense. Refer to the Task 4 quiz questions.


In [22]:
# Add your solution here
def analyse_user(uid):
#      the books that the user has previously shelved (c.f. toread_dataset)
    shelved_books = toread_iids[toread_uids == uid].tolist()
    print("*"*50)
    print("The books that user %s has previously shelved:" % uid)
    for book in shelved_books:
        print(getAuthorTitle(book))
#     the books that the user will read in the future (c.f. test_dataset)
    future_books = test_iids[test_uids == uid].tolist()
    print("*"*50)
    print("The books that user %s will read in the future:" % uid)
    for book in future_books:
        print(getAuthorTitle(book))
#     the top 10 books that the user were recommended by BPRMF
    rec_books = get_top_K(BPRMF, uid=uid, k=10)[0].tolist()
    print("*"*50)
    print("The books BPRMF recommended for user %s" % uid)
    for book in rec_books:
        print(getAuthorTitle(book))
#     compare books recommended with those has been shelved
    common_ac = list(set(shelved_books).intersection(set(rec_books)))
    print("*"*50)
    print("Recommended books which have been shelved:")
    for book in common_ac:
        print(getAuthorTitle(book))
#     compare books recommended with those will be read in the future
    common_bc = list(set(future_books).intersection(set(rec_books)))
    print("*"*50)
    print("Recommended books which will be read in the future:")
    for book in common_bc:
        print(getAuthorTitle(book))

In [23]:
BPR_rr[1805]

0.05555555555555555

In [24]:
# uid 1805 (u336) and uid 179 (user u1331)
analyse_user(1805)

**************************************************
The books that user 1805 has previously shelved:
Stieg Larsson, Reg Keeland / The Girl Who Kicked the Hornet's Nest (Millennium, #3)
Suzanne Collins / Mockingjay (The Hunger Games, #3)
Dennis Lehane / Shutter Island
Suzanne Collins / Catching Fire (The Hunger Games, #2)
Paula Hawkins / The Girl on the Train
Robert Ludlum / The Bourne Supremacy (Jason Bourne, #2)
John Grisham / The Client
Thomas Harris / The Silence of the Lambs  (Hannibal Lecter, #2)
Daphne du Maurier, Sally Beauman / Rebecca
Robert Ludlum / The Bourne Identity (Jason Bourne, #1)
Robert Galbraith, J.K. Rowling / The Cuckoo's Calling (Cormoran Strike, #1)
Stephen King / Misery
Michael Crichton / Jurassic Park (Jurassic Park, #1)
Robert Ludlum / The Bourne Ultimatum (Jason Bourne, #3)
Stephen King, Bernie Wrightson / The Stand
Michael Crichton / The Andromeda Strain
Thomas Harris / Red Dragon (Hannibal Lecter, #1)
Lee Child / Die Trying (Jack Reacher, #2)
Lee Child / Wor

In [25]:
analyse_user(179)

**************************************************
The books that user 179 has previously shelved:
Dan Brown / Angels & Demons  (Robert Langdon, #1)
Suzanne Collins / The Hunger Games (The Hunger Games, #1)
Antoine de Saint-Exupéry, Richard Howard, Dom Marcos Barbosa, Melina Karakosta / The Little Prince
Truman Capote / Breakfast at Tiffany's
Dan Brown / The Da Vinci Code (Robert Langdon, #2)
Laura Ingalls Wilder, Garth Williams / Little House on the Prairie (Little House, #2)
Milan Kundera, Michael Henry Heim / The Unbearable Lightness of Being
Suzanne Collins / Catching Fire (The Hunger Games, #2)
John Grisham / The Client
J.R.R. Tolkien / The Lord of the Rings (The Lord of the Rings, #1-3)
J.R.R. Tolkien / The Hobbit
Margaret Mitchell / Gone with the Wind
Neil Gaiman / Stardust
Laura Ingalls Wilder, Garth Williams / Little House in the Big Woods (Little House, #1)
Pearl S. Buck / The Good Earth (House of Earth, #1)
Dan Brown / Digital Fortress
Daniel Keyes / Flowers for Algernon
Nei

# Part-C. Diversity of Recommendations

This part of the exercise is concerned with diversification, as covered in Lecture 11.

## Task 5. Measuring Intra-List Diversity


For the BPR implicit factorisation model, implement the Intra-list diversity measure (see Lecture 11) of the top 5 scored items based on their item embeddings in the `BPRMF` model. 

Implement your ILD as a function with the specification:
```python
def measure_ild(top_books : Sequence[int], K : int=5) -> float
```
where:
 - `top_books` is a list or a Numpy array of iids that have been returned for a particular user. For instance, it can be obtained from `get_top_K()`.
 - `K` is the number of top-ranked items to consider from `top_books`. 
 - Your implementation should use the item emebddings stored in the `BPRMF` model.

Calculate the ILD (with k=5). Using your code for Task 4, identify the books previously shelved and recommended for the specific users requested in the quiz, and use these to analyse the recommendations.

Hints:
 - As can be seen in `get_top_K()`, item embeddings can be obtained from `BPRMF._net.item_embeddings.weight[iid]`.
 - For obtaining the cosine similarity of PyTorch tensors, use `nn.functional.cosine_similarity(, , axis=0)`.


In [26]:
# Add your solution here
import torch.nn as nn
import torch

def measure_ild(top_books : Sequence[int], K : int=5) -> float:
#     get embeddings for each iid
    embeddings = [BPRMF._net.item_embeddings.weight[iid] for iid in top_books]
    distances = []
    for i in range(K):
        for j in range(i + 1, K):
#             calculate cosine similarity between each pair of i and j
            sim = nn.functional.cosine_similarity(embeddings[i], embeddings[j], axis=0)
#             calculate pairwise distance
            distances.append(1 - sim.detach().numpy())
    ILD = (2 / (K * (K - 1))) * np.sum(distances)
    return ILD

In [27]:
measure_ild(get_top_K(BPRMF, uid=1805, k=5)[0].tolist(), K=5)

0.7484900385141373

In [28]:
for book in get_top_K(BPRMF, uid=1805, k=5)[0].tolist():
    print(getAuthorTitle(book))

Suzanne Collins / The Hunger Games (The Hunger Games, #1)
Dan Brown / The Da Vinci Code (Robert Langdon, #2)
Dan Brown / The Lost Symbol (Robert Langdon, #3)
Michael Crichton / Disclosure
George R.R. Martin / A Clash of Kings  (A Song of Ice and Fire, #2)


In [29]:
measure_ild(get_top_K(BPRMF, uid=179, k=5)[0].tolist(), K=5)

0.27872802019119264

In [30]:
for book in get_top_K(BPRMF, uid=179, k=5)[0].tolist():
    print(getAuthorTitle(book))

John Grisham / The Partner
John Grisham / The Pelican Brief
John Grisham / The Client
John Grisham / The Brethren
John Grisham / The Street Lawyer


## Task 6. Implement MMR Diversification 

Develop an Maximal Marginal Relevance (M**M**R) diversification technique, to re-rank the top-ranked recommendations for a given user.

Your function should adhere to the specification as follows:
```python
def mmr(iids : Sequence[int], scores : Sequence[float], embs : np.ndarray, alpha : float) -> Sequence[int]:
```

where iids is a list of iids, scores are their corresponding scores (in descending order), embs is their embeddings, and alpha controls the diversification tradeoff. The function returns a re-ordering of iids. As in previous Exercises, type hints are provided for clarity; a Sequence can be a list or numpy array. 

Hints:
 - As above, for obtaining the cosine similarity of PyTorch tensors, use nn.functional.cosine_similarity(, , axis=0).

To use your `mmr()` function, provide it with the outputs of `get_top_K()`. For example, to obtain an MMR reordering of the top 10 predictions of uid 0, we can run:
```
mmr( *get_top_K(bprmodel, 0, 10), 0.5)
```

Thereafter, we provide test cases for your MMR implementation, which you  should report in the quiz. We also ask for the ILD values before and after the application of MMR.


In [31]:
from typing import Sequence
def mmr(iids : Sequence[int], scores : Sequence[float], embs : np.ndarray, alpha : float) -> Sequence[int]:

    assert len(iids) == len(scores)
    assert len(iids) == embs.shape[0]
    assert len(embs.size()) == 2
    
    iteration = len(iids)
    iids = iids.tolist() if not isinstance(iids, list) else iids
    scores = scores.tolist() if not isinstance(scores, list) else scores
    rtr_iids = []
    rtr_embs = []

#     the overall iterations
    for i in range(iteration):
#      save all the results for elements in R in one iteration to calculate MMR 
        mmr_list = []
#      iterate in R (iids)
        for itemi in range(len(iids)):
#         a list of similarity for item i and all elements in S
            cos_sims = []
#             compare cosine similarity between item i in R and item j in S
            for itemj in range(len(rtr_iids)):
                cos_sim = nn.functional.cosine_similarity(embs[itemi], rtr_embs[itemj], axis=0)
                cos_sims.append(cos_sim.detach().numpy())
#          compute for each item currently in R
            mmr_itemi = (alpha * scores[itemi] - (1 - alpha) * max(cos_sims)) if len(cos_sims) > 0 else (alpha * scores[itemi])
            mmr_list.append(mmr_itemi)
#      get the index of max value, that is the index that will be removed from R and added to S
        idx = np.argmax(mmr_list)
        rtr_iids.append(iids[idx])
        rtr_embs.append(embs[idx])
        iids.remove(iids[idx])
        scores.remove(scores[idx])
        embs = embs[torch.arange(embs.size(0)) != idx]
        
  #input your solution here returns a re-ordering of iids, such that the first ranked item is first in the list

    return rtr_iids

In [32]:
def run_MMR_testcases(mmrfn):
  example_embeddings1 = torch.tensor([[1.0,1.0],[1.0,1.0],[0,1.0],[0.1, 1.0]])
  example_embeddings2 = torch.tensor([[1.0,1.0],[1.0,1.0],[0.02,1.0],[0.01,1.0]])
  print("Testcase 0 : %s" % mmrfn([1,2,3,4], [0.5, 0.5, 0.5, 0.5],  example_embeddings1, 0.5)[0] )
  print("Testcase 1 : %s" % mmrfn([1,2,3,4], [0.5, 0.5, 0.5, 0.5],  example_embeddings1, 0.5)[1] )
  print("Testcase 2 : %s" % mmrfn([1,2,3,4], [4, 3, 2, 1],  example_embeddings1, 1)[1] )
  print("Testcase 3 : %s" % mmrfn([1,2,3,4], [0.99, 0.98, 0.97, 0.001],  example_embeddings2, 0.001)[1] )
  print("Testcase 4 : %s" % mmrfn([1,2,3,4], [0.99, 0.98, 0.97, 0.001],  example_embeddings2, 0.5)[1] )

run_MMR_testcases(mmr)

Testcase 0 : 1
Testcase 1 : 3
Testcase 2 : 2
Testcase 3 : 4
Testcase 4 : 3


Now we can analyse the impact of our MMR implementation. Let's consider again uid 179 (user u1331). 

Apply MMR on the top 10 results obtained from the BPRMF model using `get_top_K()`, with an alpha value of 0.5. The following code should help:
```python
mmr( *get_top_K(bprmodel, 179, 10), 0.5)
```

Finally, anayse the returned books. Calculate the ILD (with `k=5`), and examine the authors and titles (using `getAuthorTitle()`). 

Now answer the questions in Task 6 of the Moodle quiz.


In [33]:
#add your solution here
print(measure_ild(mmr( *get_top_K(BPRMF, 179, 10), 0.5), K=5))
for book in mmr( *get_top_K(BPRMF, 179, 10), 0.5):
    print(book)
    print(getAuthorTitle(book))

0.5566441953182221
89
John Grisham / The Partner
9
J.K. Rowling, Mary GrandPré / Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
88
John Grisham / The Street Lawyer
391
John Grisham / The Pelican Brief
906
John Grisham / The Brethren
62
John Grisham / The Rainmaker
934
John Grisham / The Runaway Jury
86
John Grisham / The Broker
92
John Grisham / The Client
91
John Grisham / The King of Torts


# Task 7

This task is not a practical task - instead there are questions that tests your understanding of some related content of the course in the quiz.

# End of Exercise

As part of your submission, you should complete the Exercise 3 quiz on Moodle.
You will need to upload your notebook, complete with the **results** of executing the code.

In [10]:
# Unit centring
a = np.array([5,5,4,3,4,5,5,4,3])

a / np.linalg.norm(a)

array([0.38807526, 0.38807526, 0.31046021, 0.23284516, 0.31046021,
       0.38807526, 0.38807526, 0.31046021, 0.23284516])

In [13]:
# Mean centring
b = np.array([5,5,4,3,4,5,5,4,3])

b - b.mean()

array([ 0.77777778,  0.77777778, -0.22222222, -1.22222222, -0.22222222,
        0.77777778,  0.77777778, -0.22222222, -1.22222222])

In [15]:
# Z score normalisation

(b - b.mean()) / np.std(b)

array([ 0.98994949,  0.98994949, -0.28284271, -1.55563492, -0.28284271,
        0.98994949,  0.98994949, -0.28284271, -1.55563492])